In [25]:
from quantities import UnitQuantity, m, mm, pF, ohm, Hz, MHz, s, degree, GHz

import numpy as np
from numpy import pi, sqrt

from sympy import latex
from IPython.display import display, Math

import skrf as rf
from scipy.optimize import minimize


def Units(x, units):
    x.units = units
    return x

def split_off_units(x, decimal_places = None):
    if decimal_places is not None:
        x = round(x, decimal_places)
    x = str(x)
    return f'{x.split()[0]} \\text{{{x.split()[1]} }}'


light_speed = 299792458 * m/s

j = 1j

Single shunt stub matching problem

In [8]:
f = 2.5*GHz
Y_L = Units(1/(4*ohm) + 2*pi*f*15.915*pF*j, 'mS')
Z_0 = 50*ohm

Z_L = Units(1/Y_L, ohm)

Z_L

array(2.00006212-2.j) * ohm

In [22]:
# Frequency, wavenumber and transmission line media
freq = rf.Frequency(start=1.5, stop=3.5, npoints=301, unit='GHz')
k = freq.w / rf.c #angular frequency over speed of light
line = rf.DefinedGammaZ0(freq, gamma= k*j, z0=Z_0)

def resulting_network(theta_delay, theta_stub):
    """
    Return a loaded single stub matching network

    NB: theta_delay and theta_stub lengths are in deg
    """
    delay_load = line.delay_load(rf.zl_2_Gamma0(Z_0, Z_L), theta_delay)
    shunted_stub = line.shunt_delay_open(theta_stub)
    return shunted_stub ** delay_load

def optim_fun(x):
    return resulting_network(x[0], x[1])['2GHz'].s_mag.ravel()

x0 = (50, 50)
bnd = (0, 180)
theta_delay, theta_stub = minimize(optim_fun, x0, bounds=(bnd, bnd)).x

line_delay_length = Units(theta_delay*degree*light_speed/(360*degree*f), 'cm')
stub_length = Units(theta_stub*degree*light_speed/(360*degree*f), 'cm')

display(Math(f'\\theta_{{\\text{{delay}}}} = {round(theta_delay, 10)} ^\\circ' ))
display(Math(f'\\theta_{{\\text{{stub}}}}={round(theta_stub, 10)} ^\\circ'))

display(Math(f'\\text{{Distance between stub and load}} = {split_off_units(round(line_delay_length, 3))}'))
display(Math(f'\\text{{Stub length}} = {split_off_units(round(stub_length, 3))}'))

Electrical_length = 151.4*degree
v_p = light_speed

Lambda = UnitQuantity('lambda', light_speed/f)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
abs(Z_L)

array(2.82847105) * ohm

In [23]:
x0 = (40, 40)
theta_delay, theta_stub = minimize(optim_fun, x0, bounds=(bnd, bnd)).x

line_delay_length = Units(theta_delay*degree*light_speed/(360*degree*f), 'cm')
stub_length = Units(theta_stub*degree*light_speed/(360*degree*f), 'cm')

display(Math(f'\\theta_{{\\text{{delay}}}} = {round(theta_delay, 10)} ^\\circ' ))
display(Math(f'\\theta_{{\\text{{stub}}}}={round(theta_stub, 10)} ^\\circ'))

display(Math(f'\\text{{Distance between stub and load}} = {split_off_units(round(line_delay_length, 3))}'))
display(Math(f'\\text{{Stub length}} = {split_off_units(round(stub_length, 3))}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Rectangular waveguide problem

In [26]:
a = 58.166*mm
d = 49.056*mm

f_101 = (3E8*(m/s)/2)*sqrt(1/a**2 + 1/d**2)
Units(f_101, GHz)

array(4.00000707) * GHz